In [1]:
# Import useful packages and pre-defined helper functions
import os
import torch
import torch.nn as nn
import dill

import utilities as utilities
import params as params
import networks as networks

# Model Activation Functions
activation_functions = [
    nn.ReLU,
    nn.Sigmoid,
    nn.SiLU,
    nn.Tanh,
    nn.ELU,
    nn.Hardsigmoid,
    nn.GELU,
]

# Model Optimizer
model_optimizer = torch.optim.SGD

# Model Loss Function
model_lossfn = nn.BCELoss

try:
    file_location = os.path.dirname(__file__)
except NameError:
    file_location = os.getcwd()
print(file_location)
out_folder = os.path.join(file_location, "figures")
result_folder = os.path.join(file_location, "result")
os.makedirs(out_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

loss_values = []


# Set parameters
for dataset_type, dataset_kwargs in params.dataset_kwargs_dict.items():
    out_folder_data = os.path.join(out_folder, dataset_type)
    os.makedirs(out_folder_data, exist_ok=True)

    # Load dataset (X, y)
    X, y = dataset_kwargs["get_data_function"](**dataset_kwargs)

    # Load to torch tensor
    # Train test split
    (
        train_dataloader,
        test_dataloader,
        train_data,
        test_data,
        X_train,
        y_train,
        X_test,
        y_test,
    ) = utilities.get_dataloader_from_numpy_dataset(
        X, y, color_label_dict=dataset_kwargs["color_label_dict"], fig_show=False
    )

    # Load to torch tensor
    # Train test split
    (
        train_dataloader,
        test_dataloader,
        train_data,
        test_data,
        X_train,
        y_train,
        X_test,
        y_test,
    ) = utilities.get_dataloader_from_numpy_dataset(
        X,
        y,
        color_label_dict=params.color_label_dict,
        fig_save=False,
        fig_fname=os.path.join(out_folder_data, f"{dataset_type}_dataset.png"),
    )

    # Get dim of data
    input_dim = X.shape[-1]
    if len(y.shape) == 1:
        output_dim = 1
    else:
        output_dim = y.shape[-1]

    for ode_varaint, ode_variant_kwargs in params.ode_kwargs_dict.items():
        for (
            solver_method,
            solver_method_kwargs,
        ) in params.solver_method_kwargs_dict.items():
            # Training
            setup_kwargs = {
                "ModelODE": networks.NeuralODE,
                "model_lossfn": model_lossfn,
                "model_optimizer": model_optimizer,
                "n_epochs": params.n_epochs,
            }

            data_kwargs = {
                "train_dataloader": train_dataloader,
                "test_dataloader": test_dataloader,
                "X_train": X_train,
                "y_train": y_train,
                "X_test": X_test,
                "y_test": y_test,
            }

            plotting_kwargs = {
                # "fig_show": True,
                "fig_save": False,
                "fig_fname": os.path.join(
                    out_folder_data, f"{ode_varaint}_{solver_method}_{dataset_type}.png"
                ),
            }
            model_object_dict = utilities.run_model(
                activation_functions=activation_functions,
                **ode_variant_kwargs,
                **solver_method_kwargs,
                **setup_kwargs,
                **data_kwargs,
                **dataset_kwargs,
                **plotting_kwargs,
            )

            # Plot transformation of X points
            for activation_function in activation_functions:
                act_name = activation_function.__name__
                plot_model = model_object_dict[act_name]

                # Save the file
                dill.dump(
                    plot_model,
                    file=open(
                        os.path.join(
                            result_folder,
                            f"{ode_varaint}_{solver_method}_{dataset_type}_{act_name}_class.pickle",
                        ),
                        "wb",
                    ),
                )

/Users/martineh/Documents/Code/Fag/MA8404/ODESolver (project)
Shape of synthetic dataset (X, y): (torch.Size([4000, 2]), torch.Size([4000]))
Shape of synthetic dataset (X, y): (torch.Size([4000, 2]), torch.Size([4000]))

Activation function ReLU:

----> EPOCH 0 of 200:


/Users/martineh/anaconda3/envs/resnet/lib/python3.9/site-packages/matplotlib/collections.py:1109: UserWarning: Collection without array used. Make sure to specify the values to be colormapped via the `c` argument.
  warnings.warn("Collection without array used. Make sure to "


Train loss: 60.54552 | Train accuracy: 39.34%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 1 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 2 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 3 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 4 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 5 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 6 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 7 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

----> EPOCH 8 of 200:
Train loss: 60.81250 | Train accuracy: 39.19%
Test loss: 61.25000 | Test accuracy: 38.75%

---->

FileNotFoundError: [Errno 2] No such file or directory: '/Users/martineh/Documents/Code/Fag/MA8404/ODESolver (project)/result/Shallow Standard ODE_Simple NN_Spiral_ReLU_class.pickle/wb'

In [4]:
# Reload the file
# reloaded_model = dill.load(
#     open(
#         os.path.join(
#             result_folder,
#             f"{ode_varaint}_{solver_method}_{dataset_type}_{act_name}_class.pickle",
#         ),
#         "rb",
#     )
# )